# Improving Customer Experience of British Airways - Business Objective #3 

Business objective #3 for improving customer experience of British Airways is as follows - analysing review topics for the four different seat types for a more targeted marketing approach, improving business by 10%.

In this part of the project, topic modelling of customer reviews of their flight experiences will be done to sieve up areas of concern of customers to recommend to the airlines to better those aspects for a better flight experience for their customers. This would be done separately for the four groups - (1) economy, (2) premium economy, (3) business class and (4) first class. 

In [1]:
# Importing relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wr
import re

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

wr.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from pprint import pprint

import gensim
from gensim import corpora

import string
from pathlib import Path
from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Carlin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Carlin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Reading in source files from directory

In [2]:
# Read in preprocessed_data.csv and display the first 5 lines of the data
df = pd.read_csv("preprocessed_data.csv")
df.head(10)

,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,...,Food&Beverages,InflightEntertainment,Wifi&Connectivity,CleanText,SpecialCharacters,Tokens,HyphenatedWords,Sentiment,Bigrams,Trigrams
0,1.0,"""Service level far worse then Ryanair""",L Keele,2023-11-19,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,2023-11-01,...,NaN,NaN,NaN,hour takeoff received mail state cryptic messa...,[],"['hour', 'takeoff', 'received', 'mail', 'state...",[],-0.045588,"[('hour', 'takeoff'), ('takeoff', 'received'),...","[('hour', 'takeoff', 'received'), ('takeoff', ..."
1,3.0,"""do not upgrade members based on status""",Austin Jones,2023-11-19,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,2023-11-01,...,1.0,2.0,2.0,recently delay bru lhr due staff shortage anno...,[],"['recently', 'delay', 'bru', 'lhr', 'due', 'st...",['world-class'],0.006944,"[('recently', 'delay'), ('delay', 'bru'), ('br...","[('recently', 'delay', 'bru'), ('delay', 'bru'..."
2,8.0,"""Flight was smooth and quick""",M A Collie,2023-11-16,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,2023-11-01,...,4.0,NaN,NaN,boarded time take age get runway due congestio...,[],"['boarded', 'time', 'take', 'age', 'get', 'run...",[],0.128030,"[('boarded', 'time'), ('time', 'take'), ('take...","[('boarded', 'time', 'take'), ('time', 'take',..."
3,1.0,"""Absolutely hopeless airline""",Nigel Dean,2023-11-16,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,2022-12-01,...,NaN,NaN,NaN,day advise cancelled ask rebook hour hour orig...,[],"['day', 'advise', 'cancelled', 'ask', 'rebook'...",[],0.113889,"[('day', 'advise'), ('advise', 'cancelled'), (...","[('day', 'advise', 'cancelled'), ('advise', 'c..."
4,1.0,"""Customer Service is non existent""",Gaylynne Simpson,2023-11-14,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,2023-11-01,...,1.0,1.0,1.0,travel lisbon dream vacation cruise portugal s...,[],"['travel', 'lisbon', 'dream', 'vacation', 'cru...",[],-0.073958,"[('travel', 'lisbon'), ('lisbon', 'dream'), ('...","[('travel', 'lisbon', 'dream'), ('lisbon', 'dr..."
5,1.0,"""I can’t imagine a worst airline""",A Narden,2023-11-12,True,Booked a flight from Bucharest to Manchester w...,Solo Leisure,Economy Class,Bucharest to Manchester via London,2023-09-01,...,1.0,1.0,NaN,book bucharest manchester layover heathrow del...,[],"['book', 'bucharest', 'manchester', 'layover',...",[],-0.243958,"[('book', 'bucharest'), ('bucharest', 'manches...","[('book', 'bucharest', 'manchester'), ('buchar..."
6,8.0,"""sufficient leg and arm room""",Graeme Boothman,2023-11-08,True,Booked online months ago and the only hitch wa...,Couple Leisure,Premium Economy,Manchester to Cape Town via London,2023-11-01,...,4.0,4.0,NaN,book online month ago hitch replacement meanin...,[],"['book', 'online', 'month', 'ago', 'hitch', 'r...",[],0.080565,"[('book', 'online'), ('online', 'month'), ('mo...","[('book', 'online', 'month'), ('online', 'mont..."
7,7.0,“crew were polite”,R Vines,2023-11-07,True,The flight was on time. The crew were polite. ...,Solo Leisure,Economy Class,Seville to London Gatwick,2023-11-01,...,3.0,NaN,NaN,time crew polite story outward europe generall...,[],"['time', 'crew', 'polite', 'story', 'outward',...",[],0.212500,"[('time', 'crew'), ('crew', 'polite'), ('polit...","[('time', 'crew', 'polite'), ('crew', 'polite'..."
8,2.0,"""Angry, disappointed, and unsatisfied""",Massimo Tricca,2023-11-05,False,"Angry, disappointed, and unsatisfied. My route...",Family Leisure,Economy Class,London Heatrow to Atlanta,2023-11-01,...,4.0,4.0,3.0,angry disappoint unsatisfied route london atla...,[],"['angry', 'disappoint', 'unsatisfied', 'route'...",[],0.024176,"[('angry', 'disappoint'), ('disappoint

# 2. Data Preparation - Preprocessing 


In addition to the earlier cleaning as a group which included (1) Case converter - convert to lower case, (2) Punctuation erasure, (3) Number filter, (4) Stop word filter, (5) N-chars set to 3: removal of tokens less than 3 chars and (6) removal of specific POS through POS tagging, further customsised preprocessing is done for business objective 3.

This will include (1) Cleaning of bigram and ngram columns for our analysis purposes, and (2) dropping of irrelevant columns and splitting the dataset by seat types (economy, premium economy, business class and first class).

In [3]:
# Dropping irrelevant columns for analysis
df_cleaned = df.drop(columns=['ReviewHeader', 'Datetime', 'ReviewBody', 'SpecialCharacters', 'HyphenatedWords', 
                              'OverallRating', 'Name', 'VerifiedReview', 'TypeOfTraveller', 'Route', 'DateFlown', 
                              'SeatComfort', 'CabinStaffService', 'GroundService', 'ValueForMoney', 'Aircraft', 
                              'Food&Beverages', 'InflightEntertainment', 'Wifi&Connectivity'])



In [4]:
# Tokenize the CLeanText column for further analysis later and create another column named 'CleanText2'

from nltk.tokenize import word_tokenize

# Tokenize the 'CleanText' column and keep results in new columns 'CleanText2'
df_cleaned['CleanText2'] = df_cleaned['CleanText'].apply(word_tokenize)

# Display the first few rows to confirm the tokenized output
print(df_cleaned[['CleanText', 'CleanText2']].head())



                                           CleanText  \
0  hour takeoff received mail state cryptic messa...   
1  recently delay bru lhr due staff shortage anno...   
2  boarded time take age get runway due congestio...   
3  day advise cancelled ask rebook hour hour orig...   
4  travel lisbon dream vacation cruise portugal s...   

                                          CleanText2  
0  [hour, takeoff, received, mail, state, cryptic...  
1  [recently, delay, bru, lhr, due, staff, shorta...  
2  [boarded, time, take, age, get, runway, due, c...  
3  [day, advise, cancelled, ask, rebook, hour, ho...  
4  [travel, lisbon, dream, vacation, cruise, port...  


In [5]:
# Defining additional stop words
additional_stop = {"economy", "premium", "class", "first", "business"}

# Removing from the CleanText2 column
df_cleaned['CleanText2'] = df_cleaned['CleanText2'].apply(lambda tokens: [word for word in tokens if word.lower() not in additional_stop])

print(df_cleaned[['CleanText', 'CleanText2']].head())


                                           CleanText  \
0  hour takeoff received mail state cryptic messa...   
1  recently delay bru lhr due staff shortage anno...   
2  boarded time take age get runway due congestio...   
3  day advise cancelled ask rebook hour hour orig...   
4  travel lisbon dream vacation cruise portugal s...   

                                          CleanText2  
0  [hour, takeoff, received, mail, state, cryptic...  
1  [recently, delay, bru, lhr, due, staff, shorta...  
2  [boarded, time, take, age, get, runway, due, c...  
3  [day, advise, cancelled, ask, rebook, hour, ho...  
4  [travel, lisbon, dream, vacation, cruise, port...  


In [6]:
# View cleaned DataFrame
df_cleaned.head(10)

,SeatType,Recommended,CleanText,Tokens,Sentiment,Bigrams,Trigrams,CleanText2
0,Economy Class,no,hour takeoff received mail state cryptic messa...,"['hour', 'takeoff', 'received', 'mail', 'state...",-0.045588,"[('hour', 'takeoff'), ('takeoff', 'received'),...","[('hour', 'takeoff', 'received'), ('takeoff', ...","[hour, takeoff, received, mail, state, cryptic..."
1,Economy Class,no,recently delay bru lhr due staff shortage anno...,"['recently', 'delay', 'bru', 'lhr', 'due', 'st...",0.006944,"[('recently', 'delay'), ('delay', 'bru'), ('br...","[('recently', 'delay', 'bru'), ('delay', 'bru'...","[recently, delay, bru, lhr, due, staff, shorta..."
2,Business Class,yes,boarded time take age get runway due congestio...,"['boarded', 'time', 'take', 'age', 'get', 'run...",0.128030,"[('boarded', 'time'), ('time', 'take'), ('take...","[('boarded', 'time', 'take'), ('time', 'take',...","[boarded, time, take, age, get, runway, due, c..."
3,Economy Class,no,day advise cancelled ask rebook hour hour orig...,"['day', 'advise', 'cancelled', 'ask', 'rebook'...",0.113889,"[('day', 'advise'), ('advise', 'cancelled'), (...","[('day', 'advise', 'cancelled'), ('advise', 'c...","[day, advise, cancelled, ask, rebook, hour, ho..."
4,Economy Class,no,travel lisbon dream vacation cruise portugal s...,"['travel', 'lisbon', 'dream', 'vacation', 'cru...",-0.073958,"[('travel', 'lisbon'), ('lisbon', 'dream'), ('...","[('travel', 'lisbon', 'dream'), ('lisbon', 'dr...","[travel, lisbon, dream, vacation, cruise, port..."
5,Economy Class,no,book bucharest manchester layover heathrow del...,"['book', 'bucharest', 'manchester', 'layover',...",-0.243958,"[('book', 'bucharest'), ('bucharest', 'manches...","[('book', 'bucharest', 'manchester'), ('buchar...","[book, bucharest, manchester, layover, heathro..."
6,Premium Economy,yes,book online month ago hitch replacement meanin...,"['book', 'online', 'month', 'ago', 'hitch', 'r...",0.080565,"[('book', 'online'), ('online', 'month'), ('mo...","[('book', 'online', 'month'), ('online', 'mont...","[book, online, month, ago, hitch, replacement,..."
7,Economy Class,yes,time crew polite story outward europe generall...,"['time', 'crew', 'polite', 'story', 'outward',...",0.212500,"[('time', 'crew'), ('crew', 'polite'), ('polit...","[('time', 'crew', 'polite'), ('crew', 'polite'...","[time, crew, polite, story, outward, europe, g..."
8,Economy Class,yes,angry disappoint unsatisfied route london atla...,"['angry', 'disappoint', 'unsatisfied', 'route'...",0.024176,"[('angry', 'disappoint'), ('disappoint', 'unsa...","[('angry', 'disappoint', 'unsatisfied'), ('dis...","[angry, disappoint, unsatisfied, route, london..."
9,Economy Class,no,infrequent flyer always first choice reassuran...,"['infrequent', 'flyer', 'always', 'first', 'ch...",0.087719,"[('infrequent', 'flyer'), ('flyer', 'always'),...","[('infrequent', 'flyer', 'always'), ('flyer', ...","[infrequent, flyer, always, choice, reassuranc..."


# 2a) Splitting the dataset by SeatType

Creating separate dataframes for each seat types for analysis.

In [7]:
def filter_and_reset(df, seat_type):
    return df[df['SeatType'] == seat_type].reset_index(drop=True)

economy_df = filter_and_reset(df_cleaned, 'Economy Class')
premium_economy_df = filter_and_reset(df_cleaned, 'Premium Economy')
business_class_df = filter_and_reset(df_cleaned, 'Business Class')
first_class_df = filter_and_reset(df_cleaned, 'First Class')

In [8]:
# test print 
print(economy_df.head())
print(premium_economy_df.head())
print(business_class_df.head())
print(first_class_df.head())

        SeatType Recommended  \
0  Economy Class          no   
1  Economy Class          no   
2  Economy Class          no   
3  Economy Class          no   
4  Economy Class          no   

                                           CleanText  \
0  hour takeoff received mail state cryptic messa...   
1  recently delay bru lhr due staff shortage anno...   
2  day advise cancelled ask rebook hour hour orig...   
3  travel lisbon dream vacation cruise portugal s...   
4  book bucharest manchester layover heathrow del...   

                                              Tokens  Sentiment  \
0  ['hour', 'takeoff', 'received', 'mail', 'state...  -0.045588   
1  ['recently', 'delay', 'bru', 'lhr', 'due', 'st...   0.006944   
2  ['day', 'advise', 'cancelled', 'ask', 'rebook'...   0.113889   
3  ['travel', 'lisbon', 'dream', 'vacation', 'cru...  -0.073958   
4  ['book', 'bucharest', 'manchester', 'layover',...  -0.243958   

                                             Bigrams  \
0  [('hour'

# 2b) Creating TF-IDF vectorizer for each SeatType

In [9]:
import pandas as pd
import gensim
from gensim import corpora
from gensim.models import TfidfModel

# Build the dictionary (CleanText2 for tokenised)
mydict = corpora.Dictionary(df_cleaned['CleanText2'])
vocab_len = len(mydict)
corpus = [mydict.doc2bow(tokens) for tokens in df_cleaned['CleanText2']]
tfidf_model = TfidfModel(corpus)

def get_tfidf_features(df_cleaned, column_name='CleanText2'):
    def process_row(tokens):
        doc = mydict.doc2bow(tokens)
        # Converting the tokens into the format that the model requires
        features = gensim.matutils.corpus2csc([tfidf_model[doc]], num_terms=vocab_len).toarray()[:,0]
        return features
    
    # Generate TF-IDF features
    return pd.DataFrame([process_row(tokens) for tokens in df_cleaned[column_name]], 
                        index=df_cleaned.index,
                        columns=[mydict[i] for i in range(vocab_len)])

# Applying TF-IDF to each seat type
def apply_tfidf_to_all_seat_types():
    tfidf_dfs = {}
    
    seat_types = {
        'Economy Class': economy_df,
        'Premium Economy Class': premium_economy_df,
        'Business Class': business_class_df,
        'First Class': first_class_df
    }
    
    for seat_type, df in seat_types.items():
        print(f"Generating TF-IDF features for {seat_type}")
        tfidf_df = get_tfidf_features(df, 'CleanText2')
        tfidf_dfs[seat_type] = tfidf_df
        print(tfidf_df.head())
    return tfidf_dfs

# Generate TF-IDFs for all seat types
tfidf_features_all = apply_tfidf_to_all_seat_types()


Generating TF-IDF features for Economy Class
    airport    anyhow     avoid       bad  capacity     check   cryptic  \
0  0.067140  0.241413  0.103781  0.065405  0.191424  0.104833  0.278865   
1  0.000000  0.000000  0.000000  0.051152  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.050328  0.000000  0.161334  0.000000   
3  0.109771  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.118266  0.000000  0.000000  0.057605  0.000000  0.000000  0.000000   

      delay  departure  disruption  ...  lhrjfklaxlhr  presuming  polystyrene  \
0  0.060731   0.084876    0.231605  ...           0.0        0.0          0.0   
1  0.189988   0.000000    0.000000  ...           0.0        0.0          0.0   
2  0.000000   0.000000    0.000000  ...           0.0        0.0          0.0   
3  0.000000   0.000000    0.000000  ...           0.0        0.0          0.0   
4  0.106976   0.000000    0.000000  ...           0.0        0.0          0.0   

   installed  gri

# 3) Creating the LDA Model for topic analysis and analysing alongside bigrams and trigrams

LDA Model is created to identify the common topics mentioned by customers for each seat type. With the top topics generated for each seat type, top bigrams/trigrams will then be analysed alongside as well. Thereafter, bigrams/trigrams generated containing the most common topcis will be generated.

# 3a) Creating LDA modelling for topic analysis of each Seat Type

Creating the LDA model for the "Economy Class" to identify top topics of interest

In [10]:
# LDA Model for Economy Class
topic_num = 20
word_num = 10

# Use the tokenized reviews in the CleanText2 column
tokenized_reviews_EC = economy_df['CleanText2']

# Creating dictionary and document-term matrix
dictionary = gensim.corpora.Dictionary(tokenized_reviews_EC)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews_EC]

# Creating the LDA model
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=topic_num, id2word=dictionary, passes=20)

# Printing the topics
pprint(ldamodel.print_topics(num_topics=topic_num, num_words=word_num))

[(0,
  '0.028*"seat" + 0.012*"cabin" + 0.011*"heathrow" + 0.010*"london" + '
  '0.010*"crew" + 0.009*"fly" + 0.009*"service" + 0.009*"time" + 0.009*"meal" '
  '+ 0.008*"food"'),
 (1,
  '0.012*"heathrow" + 0.011*"fly" + 0.011*"hour" + 0.011*"seat" + '
  '0.011*"service" + 0.010*"staff" + 0.010*"make" + 0.009*"would" + '
  '0.009*"travel" + 0.008*"london"'),
 (2,
  '0.018*"seat" + 0.011*"cabin" + 0.011*"fly" + 0.011*"service" + 0.010*"time" '
  '+ 0.010*"good" + 0.010*"food" + 0.009*"crew" + 0.008*"london" + '
  '0.008*"well"'),
 (3,
  '0.020*"get" + 0.018*"hour" + 0.014*"customer" + 0.013*"tell" + 0.012*"day" '
  '+ 0.012*"would" + 0.010*"london" + 0.010*"service" + 0.009*"book" + '
  '0.009*"back"'),
 (4,
  '0.033*"seat" + 0.020*"good" + 0.015*"crew" + 0.014*"cabin" + 0.014*"food" + '
  '0.013*"service" + 0.011*"london" + 0.010*"fly" + 0.009*"entertainment" + '
  '0.008*"great"'),
 (5,
  '0.007*"seat" + 0.007*"hour" + 0.006*"time" + 0.006*"food" + 0.005*"london" '
  '+ 0.005*"cost" + 0

# Computing perplexity and coherence scores to pick best number of topics

For this portion, I had tested for topic numbers between 5 to 30, and it was found that for topic number 20, it had the lowest scores for perplexity and highest scores for coherence. Hence, topic_num = 20 was chosen.

In [11]:
#compute perplexity 
print('Perplexity: ', ldamodel.log_perplexity(doc_term_matrix))

Perplexity:  -7.5330971965750235


In [12]:
#compute coherence 

from gensim.models.coherencemodel import CoherenceModel

Coherence_Model = CoherenceModel(model=ldamodel, corpus=doc_term_matrix,coherence='u_mass')
Coherence_Model.get_coherence()


-1.5135620403198233

Creating the LDA model for the "Premium Economy" class to identify top topics of interest

In [13]:
# LDA Model for Premium Economy Class
topic_num = 20
word_num = 10

# Use the tokenized reviews in the CleanText2 column
tokenized_reviews_PE = premium_economy_df['CleanText2']

# Creating dictionary and document-term matrix
dictionary = gensim.corpora.Dictionary(tokenized_reviews_PE)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews_PE]

# Creating the LDA model
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=topic_num, id2word=dictionary, passes=20)

# Printing the topics
pprint(ldamodel.print_topics(num_topics=topic_num, num_words=word_num))

[(0,
  '0.019*"seat" + 0.017*"service" + 0.014*"staff" + 0.012*"cabin" + '
  '0.011*"traveller" + 0.009*"would" + 0.009*"world" + 0.009*"food" + '
  '0.008*"fly" + 0.007*"time"'),
 (1,
  '0.019*"seat" + 0.013*"would" + 0.012*"food" + 0.012*"meal" + '
  '0.009*"experience" + 0.008*"good" + 0.008*"crew" + 0.007*"product" + '
  '0.007*"window" + 0.007*"get"'),
 (2,
  '0.016*"crew" + 0.014*"good" + 0.012*"food" + 0.012*"cabin" + '
  '0.012*"service" + 0.011*"drink" + 0.011*"seat" + 0.010*"offer" + '
  '0.009*"meal" + 0.009*"one"'),
 (3,
  '0.016*"seat" + 0.012*"get" + 0.011*"singapore" + 0.011*"service" + '
  '0.011*"bottle" + 0.008*"staff" + 0.008*"baby" + 0.008*"london" + '
  '0.008*"time" + 0.008*"year"'),
 (4,
  '0.030*"seat" + 0.013*"cabin" + 0.012*"world" + 0.011*"good" + '
  '0.010*"traveller" + 0.010*"get" + 0.010*"leg" + 0.010*"long" + 0.008*"meal" '
  '+ 0.008*"much"'),
 (5,
  '0.013*"get" + 0.013*"london" + 0.012*"fly" + 0.011*"seat" + 0.011*"service" '
  '+ 0.010*"book" + 0.007

Creating the LDA model for the "Business Class" to identify top topics of interest

In [14]:
# LDA Model for Business Class
topic_num = 20
word_num = 10

# Use the tokenized reviews in the CleanText2 column
tokenized_reviews_BC = business_class_df['CleanText2']

# Creating dictionary and document-term matrix
dictionary = gensim.corpora.Dictionary(tokenized_reviews_BC)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews_BC]

# Creating the LDA model
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=topic_num, id2word=dictionary, passes=20)

# Printing the topics
pprint(ldamodel.print_topics(num_topics=topic_num, num_words=word_num))

[(0,
  '0.013*"seat" + 0.010*"fly" + 0.010*"offer" + 0.009*"food" + 0.009*"club" + '
  '0.009*"staff" + 0.008*"new" + 0.008*"europe" + 0.007*"hour" + '
  '0.007*"experience"'),
 (1,
  '0.014*"seat" + 0.011*"new" + 0.011*"london" + 0.010*"even" + 0.010*"crew" + '
  '0.010*"food" + 0.010*"service" + 0.009*"cabin" + 0.009*"good" + '
  '0.008*"fly"'),
 (2,
  '0.012*"crew" + 0.011*"experience" + 0.010*"one" + 0.008*"club" + '
  '0.008*"food" + 0.007*"new" + 0.006*"world" + 0.006*"lounge" + 0.006*"hour" '
  '+ 0.006*"time"'),
 (3,
  '0.029*"seat" + 0.012*"get" + 0.008*"crew" + 0.006*"passenger" + '
  '0.006*"time" + 0.006*"ask" + 0.006*"cabin" + 0.005*"say" + 0.005*"club" + '
  '0.005*"row"'),
 (4,
  '0.010*"lounge" + 0.009*"crew" + 0.009*"service" + 0.008*"good" + '
  '0.008*"club" + 0.008*"would" + 0.007*"delay" + 0.006*"late" + 0.006*"get" + '
  '0.006*"meal"'),
 (5,
  '0.036*"seat" + 0.008*"passenger" + 0.007*"get" + 0.006*"crew" + '
  '0.006*"attendant" + 0.005*"take" + 0.005*"aisle" + 

Creating the LDA model for the "First Class" to identify top topics of interest

In [15]:
# LDA Model for First Class
topic_num = 20
word_num = 10

# Use the tokenized reviews in the CleanText2 column
tokenized_reviews_FC = first_class_df['CleanText2']

# Creating dictionary and document-term matrix
dictionary = gensim.corpora.Dictionary(tokenized_reviews_FC)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews_FC]

# Creating the LDA model
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=topic_num, id2word=dictionary, passes=20)

# Printing the topics
pprint(ldamodel.print_topics(num_topics=topic_num, num_words=word_num))

[(0,
  '0.014*"great" + 0.013*"crew" + 0.011*"food" + 0.011*"staff" + 0.010*"good" '
  '+ 0.010*"service" + 0.009*"lounge" + 0.008*"return" + 0.008*"would" + '
  '0.008*"passenger"'),
 (1,
  '0.019*"crew" + 0.013*"good" + 0.011*"seat" + 0.010*"london" + 0.008*"cabin" '
  '+ 0.008*"get" + 0.008*"day" + 0.007*"excellent" + 0.007*"two" + '
  '0.006*"make"'),
 (2,
  '0.022*"seat" + 0.012*"good" + 0.012*"crew" + 0.012*"service" + '
  '0.011*"lounge" + 0.011*"well" + 0.010*"food" + 0.010*"experience" + '
  '0.008*"heathrow" + 0.008*"club"'),
 (3,
  '0.011*"good" + 0.011*"day" + 0.011*"case" + 0.008*"make" + 0.008*"usd" + '
  '0.007*"give" + 0.007*"show" + 0.007*"club" + 0.006*"dinner" + '
  '0.006*"selection"'),
 (4,
  '0.019*"good" + 0.015*"food" + 0.013*"seat" + 0.010*"london" + '
  '0.010*"heathrow" + 0.010*"return" + 0.010*"new" + 0.008*"club" + '
  '0.008*"service" + 0.007*"crew"'),
 (5,
  '0.022*"seat" + 0.014*"cabin" + 0.013*"good" + 0.009*"world" + 0.009*"crew" '
  '+ 0.008*"excellen

# 3b) Identifying top occurring trigrams for each Seat Type based on Topic Words identified from Topic Modelling

In [21]:
import pandas as pd
from collections import Counter
from nltk import trigrams

# Generating trigrams as tokenized words (trigrams were initially in object form)
def generate_trigrams(df, column_name):
    return df[column_name].apply(lambda tokens: list(trigrams(tokens)))

# Function to filter trigrams containing a specific word
def list_trigrams_containing_word(df, column, word):
    all_trigrams = []
    for sublist in df[column]:
        if isinstance(sublist, list):
            for trigram in sublist:
                if isinstance(trigram, tuple):
                    all_trigrams.append(trigram)
    
    # Filtering trigrams with specific topic word
    filtered_trigrams = [trigram for trigram in all_trigrams if word in trigram]
    
    return filtered_trigrams


# Function to get top 20 trigrams containing the specified word
def top_trigrams_with_word(df, column, word, top_n=20):
    trigrams_with_word = list_trigrams_containing_word(df, column, word)
    trigram_counts = Counter(trigrams_with_word)
    return trigram_counts.most_common(top_n)

# Function to print top trigrams
def print_top_trigrams(trigrams, seat_type, word):
    print(f"Top trigrams featuring '{word}' in reviews of {seat_type}:")
    for trigram, count in trigrams:
        print(f"{trigram}: {count}")

# Print trigrams for each seat type
economy_df['Trigrams'] = generate_trigrams(economy_df, 'CleanText2')
premium_economy_df['Trigrams'] = generate_trigrams(premium_economy_df, 'CleanText2')
business_class_df['Trigrams'] = generate_trigrams(business_class_df, 'CleanText2')
first_class_df['Trigrams'] = generate_trigrams(first_class_df, 'CleanText2')



In [25]:
# Set the commonly occurring topic word-of-interest
word = 'food' 

# Print top 20 trigrams with count that contain the frequently occurring topic word
economy_top_trigrams = top_trigrams_with_word(economy_df, 'Trigrams', word)
print_top_trigrams(economy_top_trigrams, "Economy Class", word)

premium_economy_top_trigrams = top_trigrams_with_word(premium_economy_df, 'Trigrams', word)
print_top_trigrams(premium_economy_top_trigrams, "Premium Economy", word)

business_class_top_trigrams = top_trigrams_with_word(business_class_df, 'Trigrams', word)
print_top_trigrams(business_class_top_trigrams, "Business Class", word)

first_class_top_trigrams = top_trigrams_with_word(first_class_df, 'Trigrams', word)
print_top_trigrams(first_class_top_trigrams, "First Class", word)


Top trigrams featuring 'food' in reviews of Economy Class:
('charge', 'food', 'drink'): 9
('complimentary', 'food', 'drink'): 7
('free', 'food', 'drink'): 7
('pay', 'food', 'drink'): 7
('food', 'drink', 'service'): 7
('food', 'pretty', 'good'): 7
('purchase', 'food', 'drink'): 5
('buy', 'food', 'drink'): 5
('food', 'drink', 'serve'): 5
('food', 'drink', 'offer'): 4
('food', 'beverage', 'board'): 4
('entertainment', 'good', 'food'): 4
('cabin', 'crew', 'food'): 4
('food', 'good', 'service'): 4
('food', 'drink', 'short'): 4
('food', 'drink', 'good'): 3
('crew', 'friendly', 'food'): 3
('sell', 'food', 'drink'): 3
('food', 'long', 'haul'): 3
('start', 'charge', 'food'): 3
Top trigrams featuring 'food' in reviews of Premium Economy:
('service', 'good', 'food'): 2
('crew', 'excellent', 'food'): 2
('food', 'inflight', 'entertainment'): 2
('entertainment', 'system', 'food'): 2
('leg', 'room', 'food'): 2
('food', 'wine', 'good'): 2
('take', 'food', 'drink'): 2
('food', 'choice', 'run'): 2
('foo